# Goodreads Data Wrangling and Feature Extraction

In this notebook, we will do the below steps:
1. Connect to the Database and Fetch Data and create a Pandas Dataframe for further analysis
2. Data Wrangling to clean and tranform the data
3. Feature Extraction to get some new useful features for building a Model

## Step1: Fetch Data from Database

### 1.1 Import Libraries

In [1]:
import sqlite3
# Data wrangling
import pandas as pd
from collections import Counter
import re
import numpy as np
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from textblob import Word
from nltk.probability import FreqDist
#from textblob import TextBlob
st = PorterStemmer()
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oindrilasen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.2 Connect to the Database

In [2]:
conn = sqlite3.connect('/Users/oindrilasen/Notebooks/CapstoneProject1/Final/goodreads_db.db.sqlite')
cur = conn.cursor()
print(conn)
print(cur)

### 1.3 Fetch Data from Database into pandas Dataframe

In [3]:
df_books = pd.read_sql_query("select * from books order by book_id",conn)
df_authors = pd.read_sql_query("select * from authors order by book_id",conn)
print("Records and Columns in Books DF: ",df_books.shape)
print("Records and Columns in Authors DF: ",df_authors.shape)

Records and Columns in Books DF:  (9576, 11)
Records and Columns in Authors DF:  (9576, 9)


### 1.4 Merge the books and authora data

In [4]:
df_details = pd.merge(df_books, df_authors)
print("Total Records and Features: ",df_details.shape)

Total Records and Features:  (9576, 19)


### 1.5 Explore Data

In [5]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9576 entries, 0 to 9575
Data columns (total 19 columns):
book_id             9576 non-null int64
isbn                7864 non-null object
title               9576 non-null object
total_pages         8848 non-null float64
average_rating      9576 non-null float64
ratings_count       9576 non-null int64
reviews_count       9576 non-null int64
publication_date    8029 non-null object
publisher           8521 non-null object
popular_shelves     8029 non-null object
book_description    9370 non-null object
author_id           9576 non-null int64
author_name         9576 non-null object
birth_on            3482 non-null object
death_on            1201 non-null object
fans_count          9576 non-null int64
gender              8321 non-null object
hometown            5729 non-null object
works_count         9576 non-null int64
dtypes: float64(2), int64(6), object(11)
memory usage: 1.5+ MB


In [6]:
df_details.head(2)

,book_id,isbn,title,total_pages,average_rating,ratings_count,reviews_count,publication_date,publisher,popular_shelves,book_description,author_id,author_name,birth_on,death_on,fans_count,gender,hometown,works_count
0,1,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,652.0,4.56,1940880,26187,"9,16,2006",Scholastic Inc.,"to-read,fantasy,favorites,currently-reading,yo...",When Harry Potter and the Half-Blood Prince op...,1077326,J.K. Rowling,1965/07/31,None,198576,female,"Yate, South Gloucestershire, England",191
1,2,0439358078,Harry Potter and the Order of the Phoenix (Har...,870.0,4.49,1993215,27553,"9,1,2004",Scholastic Inc.,"to-read,fantasy,favorites,currently-reading,yo...",There is a door at the end of a silent corrido...,1077326,J.K. Rowling,1965/07/31,None,198576,female,"Yate, South Gloucestershire, England",191


## Step2: Data Wrangling

### 2.1 Convert the Gender column to category

In [7]:
df_details.gender=df_details.gender.astype('category')

### 2.2 Convert the Date columns to Dates

In [8]:
df_details['birth_on'] = pd.to_datetime(df_details["birth_on"].str.strip(), format='%Y/%m/%d',errors='coerce')
df_details['death_on'] = pd.to_datetime(df_details["death_on"].str.strip(), format='%Y/%m/%d', errors='coerce')
df_details['publication_date'] = pd.to_datetime(df_details['publication_date'].str.strip(), format='%m,%d,%Y', errors='coerce')

### 2.3 Check for NULL columns

In [9]:
df_details.isnull().sum()

book_id                0
isbn                1712
title                  0
total_pages          728
average_rating         0
ratings_count          0
reviews_count          0
publication_date    1547
publisher           1055
popular_shelves     1547
book_description     206
author_id              0
author_name            0
birth_on            6104
death_on            8382
fans_count             0
gender              1255
hometown            3847
works_count            0
dtype: int64

### 2.4 Handling Missing Data

In [10]:
# Handle missing data for total_pages
df_details["total_pages"] = df_details["total_pages"].fillna(round(df_details["total_pages"].mean()))

In [11]:
# Handle missing data for fans_count
df_details["fans_count"] = df_details["fans_count"].fillna(round(df_details["fans_count"].mean()))

In [12]:
# Handle missing data for popular_shelves
df_details['popular_shelves'] = df_details['popular_shelves'].fillna("No_Tags")

In [13]:
# Handle missing data for gender
df_details["gender"] = df_details["gender"].ffill()

In [14]:
# Handle missing data for book_description
df_details["book_description"] = df_details["book_description"].fillna("No_Description")

## Step 3: Feature Extraction

### 3.1 Fetch popular tags for books

In [15]:
## Convert the series to list
shelves = df_details["popular_shelves"].values.tolist()
# Join all the tags
#shelves = ','.join(filter(None,shelves))
shelves = ','.join(shelves)
shelves = shelves.split(",") 

In [16]:
# Exclude NOT-SO-VALID Tags
exclude_item = ['fiction-fantasy','literature','science-fiction','science_fantasy','fiction','sciencefiction','science-fiction','sf','sci-fi','scifi','sff','science','to-read-sci-fi',
                'to-read','currently-reading','unread','read-in-2011','read-in-2012','read-in-2014','read-in-2015',
                'read-2016','read-in-2016','read-in-2017','read-in-2018','to-read-scifi','general-fiction','classic-sci-fi',
                're-read','not-read','reviewed','owned-but-not-read','to-read-fiction','genre-science-fiction',
                'owned','favorites','library','books-i-own','owned-books','to-buy','own-it','i-own','home-library','bookshelf',
                'novels','my-library','wish-list','books','my-books','default','have','favourites','general','maybe',
                'collection','book-club','library-book','personal-library','library-books','books-i-have','shelfari-favorites',
                'tbr','ya','want','owned-unread','and','next','reread','mine','favorite','owned-to-read','to-re-read',
                'read-in-2010','1','read-2013','shelfari-wishlist','shelved','to-get','wanted','read-2017','owned-to-read',
                'novel','need-to-buy','sci-fi-to-read','my-collection','read-2014','on-hold','all-time-favorites','my-collection','want-to-read',
                'read-2015','want-to-read','my-collection','read-2014','read-2018','read-in-2013','other','borrowed','stories','speculative-fiction',
                'finished','to-read-fantasy','to-read-owned','collections','science-fantasy','fantasy-fiction','fantasy-science-fiction',
                'fantasy-sci-fi','sci-fi-fantasy','scifi-fantasy','science-fiction-fantasy','science-fiction-and-fantasy','sf-fantasy','sf-f',
                'sci-fi-and-fantasy','fantasy-scifi','high-fantasy','fantasy-sf','fantasy','on-my-shelf','fiction-science-fiction',
                'ebook','kindle','ebooks','series','audiobook', 'audiobooks', 'adult','e-book','abandoned','adult-fiction','read-in-english',
                'young-adult','4-stars','5-stars','favorite-series','science-fiction-romance', 'hardcover', 'kindle-books', 'children','my-bookshelf',
                'audio','e-books'
               ]
# Remove all genere from shelves which are in exclude_item
shelves = [ elem for elem in shelves if elem not in exclude_item]

In [17]:
# Check top 10 genres
count = Counter(shelves)
for genre, frequency in count.most_common(5):
    print(genre,frequency)

adventure 4897
romance 4062
paranormal 3712
dnf 3438
audible 3005


In [19]:
df_details['classics']   = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'classics','classic','literary','epic'} for i in x.split(","))), True,False)
df_details['thriller']   = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'mystery','suspense','mystery-suspense','thriller','thrillers','mystery-thriller','crime','mysteries','horror-thriller','suspense-thriller','thriller-suspense','detective','romantic-suspense'} for i in x.split(","))), True,False)
df_details['romance']    = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'romance','love','paranormal-romance','romance-paranormal','erotic-romance','erotic','erotica','contemporary-romance','sci-fi-romance','scifi-romance','fantasy-romance','alien-romance','m-m-romance','love-triangle','ya-romance','adult-romance','futuristic-romance','romantic-suspense','romance-sci-fi','historical-romance'} for i in x.split(","))), True,False)
df_details['paranormal'] = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'paranormal','supernatural','vampire','vampires','vamps','zombies','zombie','witches','paranormal-fantasy','paranormal-urban-fantasy','romance-paranormal','fantasy-paranormal','ya-paranormal','adult-paranormal','paranormal-supernatural'} for i in x.split(","))), True,False)
df_details['humour']     = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'humour','funny','fun','humorous','comedy','humor'} for i in x.split(","))), True,False)
df_details['dystopian']  = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'dystopian','dystopia','dystopian-post-apocalyptic','dystopian-fiction','dystopias','distopian','distopia','dystopia-utopia','ya-dystopia','dystopian-apocalyptic'} for i in x.split(","))), True,False)
df_details['historical'] = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'history','historical','historical-fiction','historical-fantasy','historical-romance'} for i in x.split(","))), True,False)
df_details['comics']     = np.where(df_details['popular_shelves'].apply(lambda x: sum(i in {'comics','comic','comix','comic-fantasy','graphic-novels','comics-graphic-novels','graphic','graphic-novels-comics','comic-books','comics-and-graphic-novels','graphic-novels-and-comics','comics-manga','read-comics','comics-read','comic-book','comic-graphic-novel','graphicnovels','comics-and-manga','graphic-novel-comic','comics-to-read','manga-graphic-novels','cómics','graphic-novels-comic-books','comic-books-graphic-novels','illustrated','read-graphic-novels'} for i in x.split(","))), True,False)
del df_details['popular_shelves']

### 3.2 Create Bag of words for Book Description

#### Data Preprocessing:

In [20]:
# to lower case
df_details["word_list"] = df_details["book_description"].str.lower()
# remove html tags
cleantags = re.compile('<.*?>')
df_details["word_list"] = df_details["word_list"].str.replace(cleantags,' ')
# Remove punctuations
df_details["word_list"] = df_details["word_list"].str.replace('[^\w\s]',' ')
# Remove trailing spaces
df_details["word_list"] = df_details["word_list"].str.strip()
# Remove spaces in between words
df_details["word_list"] = df_details["word_list"].str.replace(' +', ' ')
# Remove Numbers
df_details["word_list"] = df_details["word_list"].str.replace('\d+', '')
# remove stop words
stop = stopwords.words('english')
stop.extend(["classics","thriller","horror","romance","paranormal","humour","dystopian","historical"])
df_details["word_list"] =  df_details["word_list"].apply(lambda x: " ".join(x for x in x.split() if x not in stop ))


In [21]:
# Tokenization
df_details["word_list"] = df_details["word_list"].str.strip().str.split('[\W_]+')
# Stemming
df_details["word_list"] = [[st.stem(word) for word in words]for words in df_details["word_list"]]
#Lemmatization
df_details["word_list"] = [[Word(word).lemmatize() for word in words]for words in df_details["word_list"]]

Let's take a look at a clean preprocessed tokenized data!

In [23]:
df_details["book_description"][50]

"Candidia Maria Smith-Foster, an eleven-year-old girl, is unaware that she's a Homo post hominem, mankind's next evolutionary step. <br /><br />With international relations rapidly deteriorating, Candy's father, publicly a small-town pathologist but secretly a government biowarfare expert, is called to Washington. Candy remains at home.<br /><br />The following day a worldwide attack, featuring a bionuclear plague, wipes out virtually all of humanity (i.e., Homo sapiens). With her pet bird Terry, she survives the attack in the shelter beneath their house. Emerging three months later, she learns of her genetic heritage and sets off to search for others of her kind."

In [25]:
#df_details["word_list"][50]
" ".join(df_details["word_list"][50])

'candidia maria smith foster eleven year old girl unawar homo post hominem mankind next evolutionari step intern relat rapidli deterior candi father publicli small town pathologist secretli govern biowarfar expert call washington candi remain home follow day worldwid attack featur bionuclear plagu wipe virtual human e homo sapien pet bird terri surviv attack shelter beneath hous emerg three month later learn genet heritag set search other kind'

#### Convert a collection of text to a matrix of token counts

Let's go simple here:
We are using **CountVectorizer** to create a bag of words by specifying **max_df** and **min_df** arguments.
- max_df = 0.90 means "ignore terms that appear in more than 90% of the documents".
- min_df = 50 means "ignore terms that appear in less than 50 documents".

In [26]:
# Initialize a CountVectorizer object: count_vect
count_vec = CountVectorizer(analyzer='word',tokenizer=lambda doc: doc, lowercase=False, max_df = 0.90, min_df = 50)
words_vec = count_vec.fit(df_details["word_list"])
bag_of_words = words_vec.transform(df_details["word_list"])

In [27]:
tokens = count_vec.get_feature_names()
df_words = pd.DataFrame(data=bag_of_words.toarray(),columns=tokens)
df_words.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9576 entries, 0 to 9575
Columns: 2207 entries, abandon to zone
dtypes: int64(2207)
memory usage: 161.2 MB


In [28]:
df_details = pd.concat([df_details, df_words], axis=1)

### Save the Dataframe for EDA in the next section

In [29]:
%store df_details

Stored 'df_details' (DataFrame)
